# Check weird too long files

## Look at event file (I've obtained)

In [6]:
import os
import pandas as pd
import numpy as np

logfile = "/wynton/home/leng/alice-albrecht/projects/PSG_Pipeline/tests/checks_mgb_logfile.txt"
event_folder = "/wynton/group/andrews/data/PSG_Pipeline_Outputs/events/hsp_mgb/"

for line in open(logfile):
    if "events beyond 20 hours detected" in line:
        file = line.split(":")[0]
        path = os.path.join(event_folder, file)
        df = pd.read_csv(path)
        onset = df["onset"].values
        weird_idx = np.where(onset > 72000)[0]

        if len(weird_idx) > 0:
            first_weird_idx = weird_idx[0]
            print(f"{file}: first weird event at index {first_weird_idx}")
            print(df.iloc[first_weird_idx:])

hsp_mgb_ses-1_sub-S0001113512628_events.csv: first weird event at index 1170
        onset  duration  sleep_stage         event_type
1170  75226.0     0.004          NaN  recording resumed
1171  75228.0     0.004          NaN           redacted
1172  75626.0     0.004          NaN           redacted
hsp_mgb_ses-1_sub-S0001119914954_events.csv: first weird event at index 1659
        onset  duration  sleep_stage         event_type
1659  76109.0     0.004          NaN  recording resumed
1660  76112.0     0.004          NaN           redacted
1661  76141.0     0.004          NaN           redacted
hsp_mgb_ses-1_sub-S0001116466792_events.csv: first weird event at index 1192
        onset  duration  sleep_stage  \
1192  78362.0     0.004          NaN   
1193  78364.0     0.004          NaN   
1194  78385.0     0.004          NaN   
1195  79337.0     0.004          NaN   
1196  79421.0     0.004          NaN   
1197  79436.0     0.004          NaN   
1198  79548.0     0.004          NaN   
1

## Look at raw annotation file

In [23]:
import os
import pandas as pd
import numpy as np

logfile = "/wynton/home/leng/alice-albrecht/projects/PSG_Pipeline/tests/checks_mgb_logfile.txt"
mgb_folder = "/wynton/group/andrews/data/HSP/PSG/bids/MGB/"

for line in open(logfile):
    if "events beyond 20 hours detected" in line:
        file = line.split(":")[0]
        sub = file.split("_")[3]
        ses = file.split("_")[2]
        path = os.path.join(mgb_folder, sub, ses, "eeg", f"{sub}_{ses}_task-psg_annotations.csv")
        df = pd.read_csv(path)
        print("----------------{}----------------".format(file))
        print(df)
       

----------------hsp_mgb_ses-1_sub-S0001113512628_events.csv----------------
      epoch      time  duration                              event
0         1  21:48:11     0.004                    Start Recording
1         1  21:48:11    30.000                      Sleep_stage_?
2         1  21:48:11     0.004  New Montage - 1) Standard Montage
3         2  21:48:41    30.000                      Sleep_stage_?
4         2  21:48:41     0.004                           REDACTED
...     ...       ...       ...                                ...
1168    966  05:50:42     0.004                              SNORE
1169    966  05:50:47     0.004                           REDACTED
1170   2508  18:41:57     0.004                  Recording Resumed
1171   2508  18:41:59     0.004                           REDACTED
1172   2521  18:48:37     0.004                           REDACTED

[1173 rows x 4 columns]
----------------hsp_mgb_ses-1_sub-S0001119914954_events.csv----------------
      epoch      ti

## Chekc new obtained event files 

In [7]:
import os
import pandas as pd

path_folder = "/wynton/group/andrews/data/PSG_Pipeline_Outputs/events/hsp_mgb/"
selected_subjects = [ 
        # Other problem mostly light eventswithout onset
        ("S0001121717028", 1),
        ("S0001119025051", 1),
        ("S0001118345214", 1),
        ("S0001118521599", 1),
        ("S0001119179499", 1),
        ("S0001116152377", 1),
        ("S0001117087093", 1),
        ("S0001114583393", 1),
        ("S0001117612859", 1),
        ("S0001119068670", 1),
        ("S0001120007643", 1),

        # Added from your special-case psg_ids
        ("S0001113512628", 1),
        ("S0001119914954", 1),
        ("S0001114155976", 1),
        ("S0001118458443", 1),
        ("S0001116466792", 1),
        ("S0001118303341", 1),  
        ("S0001111410157", 1),
        ("S0001120974563", 1),
        ("S0001121902705", 1),
    ]
    

for file in os.listdir(path_folder):
    sub = file.split("_")[3].split("-")[1]
    ses = int(file.split("_")[2].split("-")[1])
    if (sub, ses) in selected_subjects:
        print("----------------{}----------------".format(file))
        df = pd.read_csv(os.path.join(path_folder, file))
        print(df.tail(10))

----------------hsp_mgb_ses-1_sub-S0001121717028_events.csv----------------
        onset  duration  sleep_stage             event_type
1046  25500.0    30.000          4.0                    NaN
1047  25510.0     0.005          NaN  position - supine - 1
1048  25530.0    30.000          4.0                    NaN
1049  25560.0    30.000          4.0                    NaN
1050  25590.0    30.000          4.0                    NaN
1051  25620.0    30.000          4.0                    NaN
1052  25650.0    30.000          4.0                    NaN
1053  25680.0    30.000          4.0                    NaN
1054  25710.0    30.000          4.0                    NaN
1055  25740.0    30.000          4.0                    NaN
----------------hsp_mgb_ses-1_sub-S0001111410157_events.csv----------------
        onset  duration  sleep_stage event_type
995   27240.0      30.0          4.0        NaN
996   27270.0       0.0          NaN  lights on
997   27270.0      30.0          4.0        

## Remove evnets if not in post summary mastersheet

In [28]:
import os 
import pandas as pd

mgb_mastersheet = "/wynton/group/andrews/data/PSG_Pipeline_Outputs/mastersheets/hsp_mgb_mastersheet_post_summary.csv"
mgb_mastersheet = pd.read_csv(mgb_mastersheet)
to_keep = mgb_mastersheet['events_path'].to_list()

folder = "/wynton/group/andrews/data/PSG_Pipeline_Outputs/events/hsp_mgb/"
for file in os.listdir(folder):
    path = os.path.join(folder, file)
    if path not in to_keep:
        print("Deleting:", path)
        #os.remove(path)

In [29]:
to_keep = to_keeps = [os.path.basename(p) for p in to_keep]
events = os.listdir(folder)
for f in to_keep:
    if f not in events:
        print("Event not compute:", f)

Event not compute: hsp_mgb_ses-1_sub-S0001120602693_events.csv


In [22]:
to_keep = to_keeps = [os.path.basename(p) for p in to_keep]
to_keep[:2]

['hsp_mgb_ses-1_sub-S0001118661422_events.csv',
 'hsp_mgb_ses-1_sub-S0001115893893_events.csv']

In [26]:
len(set(to_keep))

8982

In [30]:
len(os.listdir(folder))

8981

In [23]:
events = os.listdir(folder)
for f in events:
    if f not in to_keep:
        print("Not in mastersheet:", f)

In [ ]:
events = os.listdir(folder)
for f in to_keep:
    if f not in events:
        print("Event not compute:", f)

In [31]:
selected_subjects = [ 
        # Other problem mostly light eventswithout onset
        ("S0001121717028", 1),
        ("S0001118345214", 1),
        ("S0001118521599", 1),
        ("S0001119179499", 1),
        ("S0001116152377", 1),
        ("S0001117087093", 1),
        ("S0001114583393", 1),
        ("S0001117612859", 1),
        ("S0001119068670", 1),
        ("S0001120007643", 1),

        # Added from your special-case psg_ids
        ("S0001113512628", 1),
        ("S0001119914954", 1),
        ("S0001114155976", 1),
        ("S0001118458443", 1),
        ("S0001116466792", 1),
        ("S0001118303341", 1),  
        ("S0001111410157", 1),
        ("S0001120974563", 1),
        ("S0001121902705", 1),
    ]

In [36]:
events[0]

'hsp_mgb_ses-1_sub-S0001113319294_events.csv'

In [39]:
for f in selected_subjects:
    sub, ses = f
    filename = f"hsp_mgb_ses-{ses}_sub-{sub}_events.csv"
    if filename not in events:
        print("Missing event file for:", filename)

Missing event file for: hsp_mgb_ses-1_sub-S0001116152377_events.csv


## Check if small TST are true 

In [40]:
selected_subjects = [
    ("S0001117577430", 1),
    ("S0001111646810", 1),
    ("S0001117249820", 1),
    ("S0001116470473", 1),
    ("S0001114414086", 1),
    ("S0001111287462", 1),
    ("S0001116197710", 1),
    ("S0001113955046", 1),
    ("S0001113725994", 1),
    ("S0001113812852", 1),
    ("S0001121626287", 1),
    ("S0001115970535", 1),
    ("S0001121417485", 1),
    ("S0001120826389", 1),
    ("S0001115748292", 2),
    ("S0001114785694", 1),
    ("S0001121010250", 1),
    ("S0001122117229", 1),
    ("S0001114694030", 1),
    ("S0001115373386", 1),
    ("S0001117684773", 1),
    ("S0001122130257", 1),
    ("S0001113023717", 1),
    ("S0001114907291", 1),
    ("S0001113103455", 1),
    ("S0001118052934", 1),
    ("S0001115575691", 1),
    ("S0001116105979", 1),
    ("S0001115159624", 1),
    ("S0001120112292", 1),
    ("S0001122228121", 3),
    ("S0001121499998", 1),
    ("S0001112981806", 3),
    ("S0001116909871", 1),
    ("S0001113566952", 1),
    ("S0001114937907", 1),
    ("S0001114424187", 1),
    ("S0001113751885", 1),
    ("S0001118858430", 1),
    ("S0001114166220", 1),
    ("S0001117367489", 1),
    ("S0001115436417", 1),
    ("S0001115453154", 1),
    ("S0001115940924", 1),
    ("S0001118481166", 1),
    ("S0001119247512", 1),
    ("S0001117147671", 1)
]

In [59]:
import os 
import pandas as pd
import re


mgb_raw_folder = "/wynton/group/andrews/data/HSP/PSG/bids/MGB/"
for sub, ses in selected_subjects:
    file = f"sub-{sub}_ses-{ses}_task-psg_annotations.csv"
    path = os.path.join(mgb_raw_folder, f"sub-{sub}", f"ses-{ses}", "eeg", file)
    df = pd.read_csv(path)

    # Filter
    stages_pattern = "|".join(["N1", "N2", "N3", "R", "REM"])
    df_sleep = df_filtered = df[
        df['event'].str.contains("sleep", case=False, na=False) &
        df['event'].str.contains("stage", case=False, na=False) 
    ]
    df_filtered = df[
        df['event'].str.contains("sleep", case=False, na=False) &
        df['event'].str.contains("stage", case=False, na=False) &
        df['event'].str.contains(stages_pattern, case=False, na=False)
    ]
    print(len(df_filtered) / 2)

1.0
2.0
2.5
2.5
5.0
5.0
5.5
5.5
11.0
13.5
13.5
15.0
16.5
17.5
27.0
20.5
23.0
29.5
30.0
31.5
34.5
35.5
35.5
37.0
37.5
38.5
38.5
39.0
39.0
39.5
40.0
42.0
42.0
44.5
45.0
45.5
46.5
46.5
48.0
49.5
49.5
49.5
50.5
56.0
58.0
58.5
59.5


In [ ]:
df_filtered

,epoch,time,duration,event
150,575,02:19:35,0.004,16 four . desats
151,581,02:22:39,0.004,REDACTED
152,585,02:24:43,0.004,REDACTED
153,586,02:24:59,0.004,REDACTED
154,591,02:27:29,0.004,REDACTED
...,...,...,...,...
236,1022,06:03:07,0.004,NASAL BREATH
237,1022,06:03:19,0.004,COUGH
238,1023,06:03:26,0.004,SNORE
239,1023,06:03:50,0.004,REDACTED


## Check AH REM for on sub 

In [2]:
import os
events = "/wynton/group/andrews/data/HSP/PSG/bids/MGB/"
print(os.listdir(events)[0])

sub-S0001119632850


In [ ]:
import os
import pandas as pd

path = "/wynton/group/andrews/data/HSP/PSG/bids/MGB/sub-S0001119632850/ses-1/eeg/sub-S0001119632850_ses-1_task-psg_annotations.csv"
df_events = pd.read_csv(path)
df_rem = df_events[df_events["event"] == "Sleep_stage_R"]
df_rem

,epoch,time,duration,event
191,125.0,00:13:14,30.0,Sleep_stage_R
192,126.0,00:13:44,30.0,Sleep_stage_R
194,127.0,00:14:14,30.0,Sleep_stage_R
195,128.0,00:14:44,30.0,Sleep_stage_R
197,129.0,00:15:14,30.0,Sleep_stage_R
198,130.0,00:15:44,30.0,Sleep_stage_R
199,131.0,00:16:14,30.0,Sleep_stage_R
200,132.0,00:16:44,30.0,Sleep_stage_R
201,133.0,00:17:14,30.0,Sleep_stage_R
202,134.0,00:17:44,30.0,Sleep_stage_R


In [59]:
df_events = pd.read_csv(path)
df_sleep = df_events[df_events["event"].str.startswith("Sleep_stage_")]
df_sleep = df_sleep.iloc[10:].reset_index(drop=True)
df_sleep["stage"] = df_sleep["event"].str.split("_").str[2]
index_list = [115, 117, 151, 152, 154, 155, 159, 161, 169, 181, 209, 223, 225, 251, 293, 295, 298, 324, 342, 416, 419, 420, 422, 436, 443, 445, 448, 462, 500, 523]
print(df_sleep.iloc[index_list]['stage'].to_string())

115     R
117     R
151    N2
152    N1
154    N1
155     W
159     W
161    N1
169    N1
181    N1
209    N2
223    N2
225    N2
251    N3
293     R
295     R
298     R
324    N1
342    N1
416     R
419     R
420     R
422     R
436    N2
443    N1
445    N1
448    N1
462     W
500     W
523     W


In [28]:
df_pnea = df_events[df_events["event"].str.contains("pnea")]
df_pnea

,epoch,time,duration,event
1,1.0,23:11:24,10.000,RespEvent - CentralApnea - 2
193,126.0,00:13:48,28.000,RespEvent - Hypopnea - 4
196,128.0,00:14:45,32.500,RespEvent - Hypopnea - 4
240,162.0,00:31:54,19.700,RespEvent - Hypopnea - 4
242,163.0,00:32:35,25.400,RespEvent - Hypopnea - 4
245,165.0,00:33:22,22.000,RespEvent - Hypopnea - 4
247,166.0,00:34:13,15.600,RespEvent - Hypopnea - 4
256,170.0,00:36:05,14.000,RespEvent - ObstructiveApnea - 1
260,172.0,00:36:52,13.300,RespEvent - Hypopnea - 4
272,180.0,00:41:12,12.200,RespEvent - Hypopnea - 4


In [29]:
len(df_pnea)

31

In [20]:
path = "/wynton/group/andrews/data/HSP/PSG/bids/MGB/sub-S0001111189359/ses-1/eeg/sub-S0001111189359_ses-1_task-psg_annotations.csv"
df_events = pd.read_csv(path)
df_rem = df_events[df_events["event"] == "Sleep_stage_R"]
df_rem['time_sec'] = (
    pd.to_timedelta(df_rem['time']).dt.total_seconds()
)
df_rem['time_sec_diff'] = df_rem['time_sec'].diff()
df_rem[df_rem['time_sec_diff'] != 30]


/scratch/alice-albrecht/ipykernel_836855/380661452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rem['time_sec'] = (
/scratch/alice-albrecht/ipykernel_836855/380661452.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rem['time_sec_diff'] = df_rem['time_sec'].diff()


,epoch,time,duration,event,time_sec,time_sec_diff
351,160.0,23:41:27,30.0,Sleep_stage_R,85287.0,NaN
675,352.0,01:17:27,30.0,Sleep_stage_R,4647.0,-81060.0
744,409.0,01:45:57,30.0,Sleep_stage_R,6357.0,150.0
803,446.0,02:04:27,30.0,Sleep_stage_R,7467.0,1110.0
812,451.0,02:06:57,30.0,Sleep_stage_R,7617.0,60.0
817,454.0,02:08:27,30.0,Sleep_stage_R,7707.0,60.0
1094,650.0,03:46:27,30.0,Sleep_stage_R,13587.0,5880.0
1097,652.0,03:47:27,30.0,Sleep_stage_R,13647.0,60.0
1101,655.0,03:48:57,30.0,Sleep_stage_R,13737.0,90.0


In [25]:
df_pnea = df_events[df_events["event"].str.contains("pnea")]
df_pnea[70:90]

,epoch,time,duration,event
828,460.0,02:11:36,20.700,RespEvent - ObstructiveApnea - 1
837,465.0,02:14:26,21.500,RespEvent - ObstructiveApnea - 1
1089,647.0,03:45:16,21.300,RespEvent - Hypopnea - 4
1093,649.0,03:46:18,25.800,RespEvent - Hypopnea - 4
1096,651.0,03:47:06,30.200,RespEvent - Hypopnea - 4
1102,655.0,03:49:18,20.700,RespEvent - ObstructiveApnea - 1
1105,657.0,03:50:12,18.500,RespEvent - ObstructiveApnea - 1
1108,659.0,03:51:07,17.895,RespEvent - ObstructiveApnea - 1
1128,673.0,03:58:24,13.400,RespEvent - ObstructiveApnea - 1
1133,676.0,03:59:54,15.800,RespEvent - ObstructiveApnea - 1
